# Câu 1: 

In [6]:
import sqlite3
import math

# Kết nối hoặc tạo database mới
db = sqlite3.connect('faker.db')
cur = db.cursor()

# Tạo bảng nếu chưa tồn tại
cur.execute('CREATE TABLE IF NOT EXISTS data (a REAL, b REAL)')

# Dữ liệu
values = [(1, 2), (2, 4), (3, 6), (4, 8), (5, 10)]

# Thêm dữ liệu theo vòng lặp
for a_val, b_val in values:
    cur.execute('INSERT INTO data (a, b) VALUES (?, ?)', (a_val, b_val))

db.commit()


# Câu 2:

In [7]:
# Kết nối lại
conn = sqlite3.connect('faker.db')
cur = conn.cursor()

# Lấy dữ liệu từ bảng
cur.execute("SELECT a, b FROM data")
rows = cur.fetchall()

# Tách a, b
a_vals = [row[0] for row in rows]
b_vals = [row[1] for row in rows]

# Tính toán thủ công
n = len(a_vals)
sum_a = sum(a_vals)
sum_b = sum(b_vals)
sum_ab = sum(x*y for x, y in zip(a_vals, b_vals))
sum_a2 = sum(x**2 for x in a_vals)
sum_b2 = sum(y**2 for y in b_vals)

numerator = n * sum_ab - sum_a * sum_b
denominator = math.sqrt((n * sum_a2 - sum_a**2) * (n * sum_b2 - sum_b**2))

r = numerator / denominator if denominator != 0 else None

print(f"Hệ số tương quan r_AB: {r}" if r is not None else "Không thể tính hệ số tương quan.")
conn.close()


Hệ số tương quan r_AB: 1.0


# Câu 3:

In [8]:
import pandas as pd
import sqlite3
from scipy.stats import f_oneway

conn = sqlite3.connect('faker.db')
cur = conn.cursor()

# Tạo bảng nếu chưa có
cur.execute("DROP TABLE IF EXISTS car_scores")
cur.execute("""
    CREATE TABLE car_scores (
        day TEXT,
        model TEXT,
        score REAL
    )
""")

# Dữ liệu
entries = [
    ("Day 1", "A", 8), ("Day 1", "B", 9), ("Day 1", "C", 7),
    ("Day 2", "A", 7.5), ("Day 2", "B", 8.5), ("Day 2", "C", 7),
    ("Day 3", "A", 6), ("Day 3", "B", 7), ("Day 3", "C", 8),
    ("Day 4", "A", 7), ("Day 4", "B", 6), ("Day 4", "C", 5)
]

# Chèn dữ liệu
for row in entries:
    cur.execute("INSERT INTO car_scores VALUES (?, ?, ?)", row)
conn.commit()

# Truy vấn bằng pandas
df_scores = pd.read_sql_query("SELECT * FROM car_scores", conn)
groups = [group['score'].tolist() for name, group in df_scores.groupby('model')]

# ANOVA
f_stat, p_val = f_oneway(*groups)

print("One-Way ANOVA")
print(f"F-statistic: {f_stat:.4f}")
print(f"p-value: {p_val:.4f}")
print("=> Có sự khác biệt giữa các mẫu xe." if p_val < 0.05 else "=> Không có sự khác biệt đáng kể.")


One-Way ANOVA
F-statistic: 0.5495
p-value: 0.5954
=> Không có sự khác biệt đáng kể.


# Câu 4:

In [9]:
from scipy.stats import chi2_contingency

# Thêm cột phân loại
cur.execute("ALTER TABLE car_scores ADD COLUMN score_group TEXT")
cur.execute("""
    UPDATE car_scores
    SET score_group = 
        CASE
            WHEN score <= 6 THEN 'Low'
            WHEN score <= 8 THEN 'Medium'
            ELSE 'High'
        END
""")
conn.commit()

# Đọc lại
df = pd.read_sql_query("SELECT * FROM car_scores", conn)

# Tạo bảng chéo
crosstab = pd.crosstab(df['day'], df['score_group'])

print("Bảng tần suất:")
print(crosstab)

# Kiểm định
chi2, p, dof, expected = chi2_contingency(crosstab)

print("\nChi-Square Test:")
print(f"Chi2: {chi2:.4f}, p-value: {p:.4f}, DoF: {dof}")
print("Expected Frequencies:")
print(pd.DataFrame(expected, index=crosstab.index, columns=crosstab.columns))


Bảng tần suất:
score_group  High  Low  Medium
day                           
Day 1           1    0       2
Day 2           1    0       2
Day 3           0    1       2
Day 4           0    2       1

Chi-Square Test:
Chi2: 6.0952, p-value: 0.4126, DoF: 6
Expected Frequencies:
score_group  High   Low  Medium
day                            
Day 1         0.5  0.75    1.75
Day 2         0.5  0.75    1.75
Day 3         0.5  0.75    1.75
Day 4         0.5  0.75    1.75


# Câu 5:

In [10]:
from datetime import datetime

cur.execute("DROP TABLE IF EXISTS flights")
cur.execute("CREATE TABLE flights (id_time INTEGER PRIMARY KEY, departure_time INTEGER)")
cur.executemany("INSERT INTO flights (id_time, departure_time) VALUES (?, ?)", 
                [(1, 830), (2, 1445), (3, 30), (4, 1200), (5, 1650), (6, 730)])
conn.commit()

cur.execute("SELECT id_time, departure_time FROM flights")
rows = cur.fetchall()

for id_, time_ in rows:
    hour = time_ // 100
    minute = time_ % 100
    dt_obj = datetime.strptime(f"{hour}:{minute:02d}", "%H:%M")
    print((id_, time_, dt_obj.strftime("%I:%M %p").lstrip("0")))


(1, 830, '8:30 AM')
(2, 1445, '2:45 PM')
(3, 30, '12:30 AM')
(4, 1200, '12:00 PM')
(5, 1650, '4:50 PM')
(6, 730, '7:30 AM')
